# Audiobooks business case

### Steps
- Preprocess the data
- Balance the dataset
- Standardize the data
- Split the data into training, validation, and test sets. Save the newly created sets in a tensor friendly format (e.g. *.npz)

Since we are dealing with real life data, we will need to preprocess it a bit.

Note that we have removed the header row, which contains the names of the categories. We simply need the numerical data.

### Extract the data from the csv

In [17]:
import numpy as np

from sklearn.preprocessing import StandardScaler

import pickle

# Load the data
raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')

# The inputs are all columns in the csv, except for the first one (IDs) and the last one (target)
inputs = raw_csv_data[:,1:-1]
targets = raw_csv_data[:,-1]

### Balance the dataset

In [23]:
# Some Python packages could be used for balancing (e.g. SMOTE)
# Here we approach the problem manually to better understand the balancing process

# count how many targets are 1 (meaning that the customer did convert)
num_one_targets = int(np.sum(targets))

# set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0
# in order to create a "balanced" dataset, we have to remove some input/target pairs.
indices_to_remove = []

# count the number of targets that are 0. 
# once there are as many 0s as 1s, mark entries where the target is 0 for further deletion.
for i in range(targets.shape[0]):
    if targets[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# we delete all indices that we marked "to remove" in the loop above.
inputs_equal_priors = np.delete(inputs, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets, indices_to_remove, axis=0)

### Shuffle the data

In [24]:
# when the data was collected it was actually arranged by date
# shuffle the indices of the data, so the data is not arranged in any way when we feed it
shuffled_indices = np.arange(inputs_equal_priors.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = inputs_equal_priors[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Standardize the inputs

In [25]:
scaler = StandardScaler()

# Fit and transform the original data
# Note that the mean and variance remain recorded in the scaler object
scaled_inputs = scaler.fit_transform(shuffled_inputs)

### Split the dataset into train, validation, and test

In [26]:
# count the total number of samples
samples_count = scaled_inputs.shape[0]

# count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = scaled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = scaled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Test set is everything that is remaining.
test_inputs = scaled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1788.0 3579 0.49958088851634536
220.0 447 0.49217002237136465
229.0 448 0.5111607142857143


### Save the three datasets in *.npz

In [27]:
# Save the three datasets in *.npz
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

### Save the scaler

In [29]:
# Similar to how we have saved the scaler files before, we also save this scaler, so we can apply in on new data
pickle.dump(scaler, open('scaler_deep_learning.pickle', 'wb'))